# Práctica 2b

Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz.

In [ ]:
import cv2
import numpy as np

# Cargar imagen
img = cv2.imread("images/cats.png")
if img is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

h, w = img.shape[:2]

# Centro inicial: centro de la imagen
cx, cy = w / 2, h / 2

# Matriz intrínseca inicial
def get_K():
    return np.array([[w, 0, cx],
                     [0, w, cy],
                     [0, 0,  1]], dtype=np.float32)

cv2.namedWindow("Distorsión")

# Sliders (centro en 50 = coef = 0)
cv2.createTrackbar("k1", "Distorsión", 50, 100, lambda x: None)
cv2.createTrackbar("k2", "Distorsión", 50, 100, lambda x: None)
cv2.createTrackbar("p1", "Distorsión", 50, 100, lambda x: None)
cv2.createTrackbar("p2", "Distorsión", 50, 100, lambda x: None)
cv2.createTrackbar("k3", "Distorsión", 50, 100, lambda x: None)

# Callback para cambiar centro óptico con clic
def mouse_click(event, x, y, flags, param):
    global cx, cy
    if event == cv2.EVENT_LBUTTONDOWN:
        cx, cy = x, y

cv2.setMouseCallback("Distorsión", mouse_click)

while True:
    # Obtener coeficientes de sliders
    k1 = (cv2.getTrackbarPos("k1", "Distorsión") - 50) / 100.0
    k2 = (cv2.getTrackbarPos("k2", "Distorsión") - 50) / 100.0
    p1 = (cv2.getTrackbarPos("p1", "Distorsión") - 50) / 500.0
    p2 = (cv2.getTrackbarPos("p2", "Distorsión") - 50) / 500.0
    k3 = (cv2.getTrackbarPos("k3", "Distorsión") - 50) / 100.0

    dist = np.array([k1, k2, p1, p2, k3], dtype=np.float32)

    # Construir K con el centro óptico actual
    K = get_K()

    # Aplicar distorsión
    map1, map2 = cv2.initUndistortRectifyMap(K, dist, None, K, (w, h), cv2.CV_32FC1)
    distorted = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)

    # Dibujar punto del centro óptico
    display = distorted.copy()
    cv2.circle(display, (int(cx), int(cy)), 5, (0, 0, 255), -1)

    cv2.imshow("Distorsión", display)

    # Salir con Q o ESC
    key = cv2.waitKey(30) & 0xFF
    if key in [ord("q"), 27]:
        break

cv2.destroyAllWindows()


Nuevo centro óptico: (424, 188)
Nuevo centro óptico: (493, 260)
Nuevo centro óptico: (208, 262)
Nuevo centro óptico: (624, 117)
Nuevo centro óptico: (472, 246)
Nuevo centro óptico: (352, 162)
Nuevo centro óptico: (226, 180)
Nuevo centro óptico: (172, 310)
Nuevo centro óptico: (548, 351)
Nuevo centro óptico: (647, 379)
Nuevo centro óptico: (703, 256)
Nuevo centro óptico: (620, 213)
Nuevo centro óptico: (449, 184)
Nuevo centro óptico: (271, 155)
Nuevo centro óptico: (234, 95)
Nuevo centro óptico: (120, 145)
Nuevo centro óptico: (127, 271)
Nuevo centro óptico: (469, 315)
Nuevo centro óptico: (578, 244)
Nuevo centro óptico: (493, 136)
Nuevo centro óptico: (310, 147)
Nuevo centro óptico: (198, 157)
Nuevo centro óptico: (250, 315)
Nuevo centro óptico: (399, 380)
Nuevo centro óptico: (593, 345)
Nuevo centro óptico: (629, 219)
Nuevo centro óptico: (542, 129)
Nuevo centro óptico: (273, 124)
Nuevo centro óptico: (223, 164)
Nuevo centro óptico: (237, 185)
Nuevo centro óptico: (382, 178)
Nuevo cen